---
### 01. [Settings] Development Environment
---

In [6]:
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
!pip install git+https://github.com/huggingface/trl.git@7630f877f91c556d9e5a3baa4b6e2894d90ff84c

  Cloning https://github.com/huggingface/trl.git (to revision 7630f877f91c556d9e5a3baa4b6e2894d90ff84c) to /tmp/pip-req-build-phqb1wl4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-phqb1wl4
  Running command git rev-parse -q --verify 'sha^7630f877f91c556d9e5a3baa4b6e2894d90ff84c'
  Running command git fetch -q https://github.com/huggingface/trl.git 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Running command git checkout -q 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Resolved https://github.com/huggingface/trl.git to commit 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.7.12.dev0-py3-none-any.whl size=173434 sha256=4d34f8ba047c8cd43c23f2f1d14fc455ec8720bd8d51e098584477207670bfe7
  Stored in directory: /root/.cache/pip/wheels/ad/f5/b1/f5ac48230936583c88cfde8

In [ ]:
import torch
from datasets import Dataset, load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [7]:
from huggingface_hub import notebook_login
notebook_login()

---
### 02. [Load] Datasets
---

In [8]:
from datasets import load_dataset
dataset = load_dataset("zzunyang/LawQA_LawSee")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
# answer 열에서 값이 없는 행 제거
filtered_dataset = dataset.filter(lambda x: x['answer'] is not None and x['answer'].strip() != "")
filtered_dataset

Filter:   0%|          | 0/41539 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'precedent', 'answer'],
        num_rows: 41084
    })
})

In [10]:
train_test_split = filtered_dataset['train'].train_test_split(test_size=0.1)


final_dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

In [11]:
del dataset
del filtered_dataset

In [12]:
final_dataset['train'][1]

{'question': '미국국적 취득자가 한국에 입국한 후에 F4비자를 신청할 수 있나요? 한국시민\xa0이었다가\xa0최근\xa0미국국적을\xa0취득\xa0했습니다.\xa0\xa0미국여권으로\xa0이미\xa0한번\xa0한국에\xa0방문을\xa0했기\xa0때문에\xa0한국\xa0국적은\xa0박탈\xa0된\xa0상태\xa0입니다.\xa0한국에\xa0다시\xa0가족\xa0방문으로\xa0오려고\xa0하는데\xa0한국에서\xa03개월\xa0이상\xa0지내게\xa0될지\xa0아직\xa0확실치\xa0않아서\xa0그러는데\xa0F4\xa0비자를\xa0한국에\xa0입국\xa0한\xa0후에\xa0신청\xa0할수\xa0있는지\xa0궁금합니다.\xa0FBI\xa0범죄\xa0기록이\xa0필요하다는건\xa0알고\xa0있는데\xa0그건\xa0미국에서\xa0준비\xa0해서\xa0오려고\xa0합니다.',
 'precedent': '',
 'answer': '일단\xa0남자인지\xa0여자인지\xa0모르겠으나\xa0여자인\xa0경우라면\xa0가능할\xa0수\xa0있습니다.\xa0하지만\xa0남자인\xa0경우라면\xa0병역을\xa0마치지\xa0않았다면\xa0\xa0F4\xa0비자\xa0발급은\xa0일정한\xa0나이까지\xa0제한될\xa0수\xa0있습니다.\xa0이\xa0때문에\xa0한국\xa0귀국\xa0전에\xa0충분히\xa0알아보고\xa0대처하시길\xa0바랍니다.'}

In [15]:
import gc
gc.collect()

0

In [14]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

---
### 03. [Gemma Model] 법률 자문 및 문서 간소화 시스템
---

In [16]:
BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float16, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [17]:
doc = final_dataset['train']['question'][0]

In [18]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

In [20]:
messages = [
    {
        "role": "user",
        "content": "You are a Korean legal expert. Please do not repeat the same content and include only the important information. Provide the answer in full sentences and only once. Do not break it into paragraphs, and after reviewing your answer, summarize it. Write the response in Korean.:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [21]:
prompt

'<bos><start_of_turn>user\nYou are a Korean legal expert. Please do not repeat the same content and include only the important information. Provide the answer in full sentences and only once. Do not break it into paragraphs, and after reviewing your answer, summarize it. Write the response in Korean.:\n\n아파트 전세계약 만기 후 녹물에 의한 질병이 발생한 경우 어머니와\xa0공동명의의\xa0아파트\xa0한채가\xa0있습니다.거주의\xa0목적으로\xa0구매\xa0후\xa0삼천여만원의\xa0리모델링\xa0후\xa01년을\xa0넘게\xa0거주하다\xa0직장을\xa0이동하면서\xa0아파트를\xa0전세로\xa0변경\xa0내놓은\xa0부동산\xa0주인이\xa0집을\xa0마음에\xa0들어했고\xa0복비라도\xa0벌자는\xa0생각에\xa0부동산\xa0주인에게\xa0아파트를\xa0전세계약했습니다\xa0그리고\xa0천만원을\xa0올려주고\xa0계약연장을\xa0요청하여\xa0계약서\xa0재\xa0작성을\xa0요청하였으나\xa0통장의\xa0거래\xa0내역으로\xa0가능하다며\xa0계약서는\xa01차\xa0후\xa0작성하지\xa0않았습니다.그리고\xa0계약연장\xa0후\xa0집에\xa0녹물이\xa0나온다며\xa0어머니께\xa0확인해\xa0줄것을\xa0요청하였고\xa0제가\xa0거주\xa0하는\xa0동안\xa0녹물은\xa0발생하지\xa0않았지만\xa0아파트\xa0관리소에\xa0내용을\xa0통보하였습니다.그리고\xa0임차인은\xa0아무런\xa0얘기가\xa0없었으며\xa0재계약\xa0후\xa0어머니께서\xa02년\xa0후\xa0집을\xa0팔겠다고\xa0임차인(부동산중개인)에서\xa0통보하였습니다.계약\xa0만료\xa0시점인\xa02011년12월31일\

#### ⓐ Gemma-it 추론

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

#### ⓑ Gemma Fine-Tuning

In [ ]:
def generate_prompt(dataset):
    prompt_list = []
    for i in range(len(dataset['question'])):
        prompt_list.append(r"""<sos><start_of_turn>user"당신은 한국 법률 전문가입니다. 같은 내용을 반복하지 않고, 중요한 정보만 포함해 주세요. 문단 나누지 말고 문장으로 주세요.:{}<end_of_turn><start_of_turn>model{}<end_of_turn><eos>""".format(dataset['question'][i], dataset['answer'][i]))
    return prompt_list

In [ ]:
train_data = final_dataset['train']
print(generate_prompt(train_data[:1])[0])

<sos><start_of_turn>user"당신은 한국 법률 전문가입니다. 같은 내용을 반복하지 않고, 중요한 정보만 포함해 주세요. 문단 나누지 말고 문장으로 주세요.:노조위원장이 사퇴하여 부위원장이 대행 중인데 대표자 변경신고를 반드시 해야만 하나요? 대표자를 변경신고를 관청에 해야 된다고 하는데 위원장 사퇴후 부위원장이 대행하고 있습니다위원장은 선출은 3-4개월 걸릴것 같은데 변경신고를 꼭 해야 하나요?<end_of_turn><start_of_turn>model다음과 같이 변경신고의무가 존재하고 미이행시에 과태료의 대상인 것 자체는 맞습니다. 그러나 실무상 노조대표자의 변경여부를 고용노동청에서 알기가 어렵고 노조에 대한 간섭으로 비춰지기에 과태료부과는 거의 없습니다. 제13조(변경사항의 신고등) ①노동조합은 제10조제1항의 규정에 의하여 설립신고된 사항중 다음 각호의 1에 해당하는 사항에 변경이 있는 때에는 그 날부터 30일 이내에 행정관청에게 변경신고를 하여야 한다. 1. 명칭2. 주된 사무소의 소재지3. 대표자의 성명4. 소속된 연합단체의 명칭②노동조합은 매년 1월 31일까지 다음 각호의 사항을 행정관청에게 통보하여야 한다. 다만, 제1항의 규정에 의하여 전년도에 변경신고된 사항은 그러하지 아니하다. 1. 전년도에 규약의 변경이 있는 경우에는 변경된 규약내용2. 전년도에 임원의 변경이 있는 경우에는 변경된 임원의 성명3. 전년도 12월 31일 현재의 조합원수(聯合團體인 勞動組合에 있어서는 構成團體別 組合員數)<end_of_turn><eos>


---
### 04. [Settings] QLoRA
---

In [ ]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#### ⓐ Trainer 실행

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        # num_train_epochs = 1,
        max_steps=1500,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=300,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

Map:   0%|          | 0/36975 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss
100,3.243600
200,2.173800
300,2.034400
400,1.993100
500,1.936500
600,1.916400
700,1.895800
800,1.844100
900,1.829000
1000,1.817700


#### ⓑ [Save] Finetuned Model

In [ ]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-legal-ko')

---
### 05. [Gemma Model] 법률 자문 및 문서 간소화 시스템: 추론
---

In [ ]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)

In [ ]:
doc = dataset['test']['question'][10]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "관련 법률 자문 및 일반인도 이해할 수 있게 문서 간소화 해:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])